# RealEstate dataset processing

В файле "RealEstate-dataset.csv" пять признаков: Сегмент, Класс, Тип сделки, Цена и Описание. Реальные признаки только Описание и Тип сделки, остальные сгенерированы рандомно из заданного списка. Необходимо: 

1) В сегментах есть пропуски, их необходимо заполнить исходя из имеющихся классов и соотношения классов и сегментов: К сегменту Торговая недвижимость относятся классы: [ТРЦ, РТЦ, Стрит-ритейл]; к сегменту Объекты свободного назначения относится класс [ОСН]; к сегменту Офисная недвижимость относятся классы [Офисы A, Офисы B, Офисы C]; к сегменту Производственно-складская нежвимость относятся классы:[ПС A, ПС B, ПС C]. На выходе во всех сегментах должен быть правильный класс. 

2) По каждому Сегменту + Классу + Типу сделки надо посчитать две статистики: среднюю цену и медиану цены. Всего таких статистик будет: Количество Сегментов * Количество Классов * Количество Типов сделки * 2 (средняя и медиана). 

3) Необходимо заполнить признак "Отопление" (сейчас его нет). Он может принимать два значения - "Да" и "Нет". Необходимо вытащить его из Описания. Если Отопление есть в описании и не указано, что его нет, то считаем, что отопление есть (т.е. например "центральное отопление", "отопление есть" и т.д. = "Да", а если явно указано что его нет, то = "Нет"). Не обязательно все строки должны быть заполнены, где-то может и не быть никакой информации о том, есть ли отопление или его нет.

In [1]:
# Подключение библиотек
import numpy as np
import pandas as pd

In [2]:
# Пути к директориям и файлам
DATASET_PATH = 'RealEstate-dataset.csv'
PREPARED_DATASET_PATH = 'RealEstate-dataset-final.csv'

##### Смотрим содержимое датасета

In [3]:
df = pd.read_csv(DATASET_PATH)
df.head(10)

,Сегмент,Класс,Тип сделки,Описание,Цена
0,NaN,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,NaN,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81
6,NaN,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45
7,NaN,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46
8,NaN,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91
9,Производственно-складская нежвимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83


In [4]:
# Получаем информацию о датасете и пропусках
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Сегмент     7951 non-null   object
 1   Класс       10000 non-null  object
 2   Тип сделки  10000 non-null  object
 3   Описание    10000 non-null  object
 4   Цена        10000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


## Обработка пропусков

В сегментах есть пропуски, их необходимо заполнить исходя из имеющихся классов и соотношения классов и сегментов: К сегменту Торговая недвижимость относятся классы: [ТРЦ, РТЦ, Стрит-ритейл]; к сегменту Объекты свободного назначения относится класс [ОСН]; к сегменту Офисная недвижимость относятся классы [Офисы A, Офисы B, Офисы C]; к сегменту Производственно-складская нежвимость относятся классы:[ПС A, ПС B, ПС C]. На выходе во всех сегментах должен быть правильный класс.

In [5]:
# Заполняем пропуски в датасете
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'ТРЦ'), 'Сегмент'] = df['Сегмент'].fillna('Торговая недвижимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'РТЦ'), 'Сегмент'] = df['Сегмент'].fillna('Торговая недвижимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'Стрит-ритейл'), 'Сегмент'] = df['Сегмент'].fillna('Торговая недвижимость')

df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'ОСН'), 'Сегмент'] = df['Сегмент'].fillna('Объекты свободного назначения')

df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'Офисы A'), 'Сегмент'] = df['Сегмент'].fillna('Офисная недвижимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'Офисы B'), 'Сегмент'] = df['Сегмент'].fillna('Офисная недвижимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'Офисы C'), 'Сегмент'] = df['Сегмент'].fillna('Офисная недвижимость')

df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'ПС A'), 'Сегмент'] = df['Сегмент'].fillna('Производственно-складская нежвимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'ПС B'), 'Сегмент'] = df['Сегмент'].fillna('Производственно-складская нежвимость')
df.loc[(df['Сегмент'].isna()) & (df['Класс'] == 'ПС C'), 'Сегмент'] = df['Сегмент'].fillna('Производственно-складская нежвимость')

In [6]:
# Проверяем результат
df.head(30)

,Сегмент,Класс,Тип сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81
6,Производственно-складская нежвимость,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45
7,Офисная недвижимость,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46
8,Производственно-складская нежвимость,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91
9,Производственно-складская нежвимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83


In [7]:
# Проверяем, что все пропуски обработаны
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Сегмент     10000 non-null  object
 1   Класс       10000 non-null  object
 2   Тип сделки  10000 non-null  object
 3   Описание    10000 non-null  object
 4   Цена        10000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [8]:
# Сохраняем промежуточный результат
df.to_csv(PREPARED_DATASET_PATH, index=False, encoding='utf-8')

## Статистика

По каждому Сегменту + Классу + Типу сделки надо посчитать две статистики: среднюю цену и медиану цены. Всего таких статистик будет: Количество Сегментов * Количество Классов * Количество Типов сделки * 2 (средняя и медиана).

In [9]:
# Подключение библиотек
from itertools import product

In [17]:
# Загружаем промежуточный датасет
DATASET_PATH = 'RealEstate-dataset-final.csv'
df = pd.read_csv(DATASET_PATH)

In [18]:
df.head(10)

,Сегмент,Класс,Тип сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81
6,Производственно-складская нежвимость,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45
7,Офисная недвижимость,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46
8,Производственно-складская нежвимость,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91
9,Производственно-складская нежвимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83


In [19]:
# Проверим минимальное и максимальное значение по цене и заодно наличие выбросов
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Цена,10000.0,50.7252,28.289763,1.0,27.0,51.0,75.0,99.0


In [20]:
# Получаем список всех уникальных значений для Сегментов
# array(['Офисная недвижимость', 'Торговая недвижимость',
#       'Производственно-складская нежвимость',
#       'Объекты свободного назначения'], dtype=object)
a1 = df['Сегмент'].unique()

In [21]:
# Получаем список всех уникальных значений для Классов
# array(['Офисы A', 'Стрит-ритейл', 'ТРЦ', 'Офисы C', 'РТЦ', 'ПС C',
#       'Офисы B', 'ПС A', 'ОСН', 'ПС B'], dtype=object)
a2 = df['Класс'].unique()

In [22]:
# Получаем список всех уникальных значений для Типов сделки
# array(['Аренда', 'Продажа'], dtype=object)
a3 = df['Тип сделки'].unique()

In [23]:
# Статистика по цене для всех возможных комбинаций Сегментов, Классов, Типов сделки
cnt = 0
for i in list(product(a1,a2,a3)):
    cols = list(product(a1,a2,a3))
    for t in cols:
        all = len(df.loc[(df['Сегмент'] == t[0]) & (df['Класс'] == t[1]) & (df['Тип сделки'] == t[2]), 'Цена'])
        t_mean = df.loc[(df['Сегмент'] == t[0]) & (df['Класс'] == t[1]) & (df['Тип сделки'] == t[2]), 'Цена'].mean()
        t_median = df.loc[(df['Сегмент'] == t[0]) & (df['Класс'] == t[1]) & (df['Тип сделки'] == t[2]), 'Цена'].median()
        if all != 0: print(f'{cnt} - {t}, Наблюдений: {all}, Среднее: {t_mean:.2f}, Медиана: {t_median:.1f}')
        cnt += 1

0 - ('Офисная недвижимость', 'Офисы A', 'Аренда'), Наблюдений: 335, Среднее: 50.83, Медиана: 50.0
1 - ('Офисная недвижимость', 'Офисы A', 'Продажа'), Наблюдений: 56, Среднее: 49.23, Медиана: 52.0
6 - ('Офисная недвижимость', 'Офисы C', 'Аренда'), Наблюдений: 1474, Среднее: 50.67, Медиана: 51.0
7 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
12 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
13 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
22 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
23 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
24 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
25 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
28 -

320 - ('Офисная недвижимость', 'Офисы A', 'Аренда'), Наблюдений: 335, Среднее: 50.83, Медиана: 50.0
321 - ('Офисная недвижимость', 'Офисы A', 'Продажа'), Наблюдений: 56, Среднее: 49.23, Медиана: 52.0
326 - ('Офисная недвижимость', 'Офисы C', 'Аренда'), Наблюдений: 1474, Среднее: 50.67, Медиана: 51.0
327 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
332 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
333 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
342 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
343 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
344 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
345 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Меди

652 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
653 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
662 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
663 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
664 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
665 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
668 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
669 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, Медиана: 56.0
690 - ('Производственно-складская нежвимость', 'ПС C', 'Аренда'), Наблюдений: 1468, Среднее: 51.00, Медиана: 52.0
691 - ('Производственно-складская нежвимость', 'ПС C', 'Продажа'), Наблюдений: 300, 

967 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
972 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
973 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
982 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
983 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
984 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
985 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
988 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
989 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, Медиана: 56.0
1010 - ('Производственно-складская нежвимость', 'ПС C', 'Аренда'), Наблюдений: 1468, Среднее: 51.

1286 - ('Офисная недвижимость', 'Офисы C', 'Аренда'), Наблюдений: 1474, Среднее: 50.67, Медиана: 51.0
1287 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
1292 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
1293 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
1302 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
1303 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
1304 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
1305 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
1308 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
1309 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, 

...

4812 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
4813 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
4822 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
4823 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
4824 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
4825 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
4828 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
4829 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, Медиана: 56.0
4850 - ('Производственно-складская нежвимость', 'ПС C', 'Аренда'), Наблюдений: 1468, Среднее: 51.00, Медиана: 52.0
4851 - ('Производственно-складская нежвимость', 'ПС C', 'Продажа'), Наблюде

5126 - ('Офисная недвижимость', 'Офисы C', 'Аренда'), Наблюдений: 1474, Среднее: 50.67, Медиана: 51.0
5127 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
5132 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
5133 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
5142 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
5143 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
5144 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
5145 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
5148 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
5149 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, 

5447 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
5452 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
5453 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
5462 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
5463 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
5464 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
5465 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
5468 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
5469 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, Медиана: 56.0
5490 - ('Производственно-складская нежвимость', 'ПС C', 'Аренда'), Наблюдений: 1468, Сре

5767 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
5772 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
5773 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
5782 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
5783 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
5784 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
5785 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
5788 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
5789 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, Медиана: 56.0
5810 - ('Производственно-складская нежвимость', 'ПС C', 'Аренда'), Наблюдений: 1468, Сре

6086 - ('Офисная недвижимость', 'Офисы C', 'Аренда'), Наблюдений: 1474, Среднее: 50.67, Медиана: 51.0
6087 - ('Офисная недвижимость', 'Офисы C', 'Продажа'), Наблюдений: 325, Среднее: 53.45, Медиана: 56.0
6092 - ('Офисная недвижимость', 'Офисы B', 'Аренда'), Наблюдений: 333, Среднее: 50.56, Медиана: 49.0
6093 - ('Офисная недвижимость', 'Офисы B', 'Продажа'), Наблюдений: 64, Среднее: 51.72, Медиана: 54.0
6102 - ('Торговая недвижимость', 'Стрит-ритейл', 'Аренда'), Наблюдений: 1421, Среднее: 51.97, Медиана: 53.0
6103 - ('Торговая недвижимость', 'Стрит-ритейл', 'Продажа'), Наблюдений: 349, Среднее: 49.59, Медиана: 48.0
6104 - ('Торговая недвижимость', 'ТРЦ', 'Аренда'), Наблюдений: 339, Среднее: 50.15, Медиана: 51.0
6105 - ('Торговая недвижимость', 'ТРЦ', 'Продажа'), Наблюдений: 67, Среднее: 44.70, Медиана: 41.0
6108 - ('Торговая недвижимость', 'РТЦ', 'Аренда'), Наблюдений: 327, Среднее: 47.30, Медиана: 46.0
6109 - ('Торговая недвижимость', 'РТЦ', 'Продажа'), Наблюдений: 79, Среднее: 54.63, 

## Генерация фич
Необходимо заполнить признак "Отопление" (сейчас его нет). Он может принимать два значения - "Да" и "Нет". Необходимо вытащить его из Описания. Если Отопление есть в описании и не указано, что его нет, то считаем, что отопление есть (т.е. например "центральное отопление", "отопление есть" и т.д. = "Да", а если явно указано что его нет, то = "Нет"). Не обязательно все строки должны быть заполнены, где-то может и не быть никакой информации о том, есть ли отопление или его нет.

In [24]:
# Подключение библиотек
import re

In [25]:
# Загружаем промежуточный датасет
DATASET_PATH = 'RealEstate-dataset-final.csv'
df = pd.read_csv(DATASET_PATH)

In [26]:
df.head(10)

,Сегмент,Класс,Тип сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81
6,Производственно-складская нежвимость,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45
7,Офисная недвижимость,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46
8,Производственно-складская нежвимость,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91
9,Производственно-складская нежвимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83


In [27]:
# Создаем новый признак "Отопление" и заполняем все наблюдения, где упоминается про отопление
heating_all = df.Описание.str.contains('(?i)отоплен').sum()
print(f'Наблюдений, где упоминается об отоплении: {heating_all}')
df.loc[(df['Описание'].str.contains('(?i)отоплен') == True), 'Отопление'] = 'Да'

Наблюдений, где упоминается об отоплении: 10000


In [28]:
# Находим и заполняем наблюдения, где явно сказано, что отопления нет
df.loc[(df['Описание'].str.contains('(?i)нет\ отоплен|без\ отоплен|отопление\ не\ |отопления\ нет') == True), 'Отопление'] = 'Нет'
heating_no = df.Отопление.str.contains('Нет').sum()
print(f'Наблюдений, где явно указано, что нет отопления: {heating_no}')

Наблюдений, где явно указано, что нет отопления: 624


In [29]:
# Общий итог
print(f'Наблюдений, где есть отопление: {heating_all-heating_no}')
print(f'Наблюдений, где нет отопления: {heating_no}')

Наблюдений, где есть отопление: 9376
Наблюдений, где нет отопления: 624


In [30]:
# Проверяем, что все строки заполнены
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Сегмент     10000 non-null  object
 1   Класс       10000 non-null  object
 2   Тип сделки  10000 non-null  object
 3   Описание    10000 non-null  object
 4   Цена        10000 non-null  int64 
 5   Отопление   10000 non-null  object
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


In [31]:
# Проверяем результат визуально
df.head(15)

,Сегмент,Класс,Тип сделки,Описание,Цена,Отопление
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62,Да
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12,Да
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98,Да
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7,Да
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22,Да
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81,Да
6,Производственно-складская нежвимость,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45,Да
7,Офисная недвижимость,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46,Да
8,Производственно-складская нежвимость,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91,Да
9,Производственно-складская нежвимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83,Да


In [32]:
# Сохраняем финальный датасет
df.to_csv(PREPARED_DATASET_PATH, index=False, encoding='utf-8')